# gensim与LDA主题模型

In [38]:
import jieba
from gensim import corpora, models, similarities

#### 第一步：词向量化

将词转换成向量，通常有词袋模型，tfidf模型。整个过程涉及分词，过滤词汇(如低频词、停止词，标点符号等)，

In [39]:
documents = [
    "新款普拉多国内没上市，有就很多人不喜欢？",
    "丰田神车普拉多出车祸，走进仔细一看，才明白 为何普拉多叫霸道",
    "进藏神器终于大改！18款丰田普拉多实车拍摄最新消息详解",
    "18款全新普拉多中东版实拍实拍，比一汽良心太多，还是全时四驱",
    "2018款丰田普拉多，打开车门后买不买自己做决定",
    "60万的普拉多，如今20万卖出去，走进一看 这车10万都不值",
    "长城对丰田霸道又发飙了！造柴油版哈弗H9，这下普拉多认输吧！",
    "越野车只有帕杰罗、普拉多？12万售价猎豹让奥迪Q5自叹不如",
    "三分钟看懂：为什么说Teramont途昂6座版“666”",
    "以大博大，上海大众旗舰SUV Teramont途昂大步而来",
    "上汽大众Teramont定名途昂，将于三月上市",
    "无须L，看30万起步Teramont途昂如何续写神车神话",
    "大有内涵 试驾上汽大众大型SUV Teramont途昂",
    "“大”行其道，上海大众Teramont途昂城际穿越体验",
    "大众Teramont途昂最新消息 到底卖多少钱呢？",
    "Teramont途昂上市档案上汽大众旗舰大型SUV“大”揭秘",
    "请问大探2017款的导航地图是最新的吗？怎么很多地方都是导老路去？怎么升级",
    "大家好，2.3精英广州或者佛山多少价位能提到？",
    "【太平洋汽车网】庆双节·喜征文 发表任意主题精华帖 赢太平洋汽车网大礼包",
    "麻烦说一下你换下的是啥牌子的？我的韩泰申诉成功了！",
    "途昂380旗舰的高耸三指的前排头枕终于接地气了",
    "认知途昂系列之问题篇1：发动机自检异响——虚惊一场",
    "家用款启迈斯R9跑步机怎么样，这款启迈斯R9跑步机质量太差了吗",
    "全新进口大切诺基 全新智能座驾 释放无限魅力",
    "福特撼路者和福特锐界 同品牌难以选择？",
    "奔驰CLK240安装主机导航一体机+倒车影视",
    "大切3.0汽油车用什么标号机油，什么牌子取代？",
    "想给撼路者做原车屏升级，都有哪些品牌？",
    "老婆竟然在自家撼路者里叫别的男人名字",
    "探险者2.3T启动后能听到细微的类似高频电流滋滋的声音。求解",
]


In [40]:
def texts():
    for s in documents:
        words = [w.strip() for w in jieba.cut(s) if len(w.strip()) > 1]
        yield words    

corpora.Dictionary会给文档中的所有词(去重后)贴上一个唯一的整数id。该类的参数接收一个生成器（迭代器），这在实际应用中很重要，实际运用中，文档通常很大，都放内存机器会吃不消，而如果用迭代器，则可以很好处理大数据问题。
- 通过save方法，生成的字典可以序列化到硬盘上，用于后续的使用
- doc2bow可以将新分词后的分档转换成向量，这个操作中，没出现在dictionary中的词会被忽略

In [51]:
dictionary = corpora.Dictionary(texts()) 

dictionary.save('/tmp/deerwester.dict') #　保存字典，以后可以直接读取这个字典

new_s = "刚提了辆普拉多, 大家觉得怎么样"
#将分词后的文档转换成向量，　
new_vec = dictionary.doc2bow(jieba.cut(new_s))  # 同样是可以传入迭代器

In [50]:
print(new_vec)

[(5, 1), (109, 1), (147, 1)]


In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts()]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # 保存在硬盘上

In [42]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [45]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=10)

In [46]:
model.print_topics(10)

[(0,
  '0.005*"途昂" + 0.005*"普拉多" + 0.005*"Teramont" + 0.005*"品牌" + 0.005*"SUV" + 0.005*"升级" + 0.005*"大众" + 0.005*"上市" + 0.005*"旗舰" + 0.005*"多少"'),
 (1,
  '0.060*"途昂" + 0.048*"Teramont" + 0.037*"旗舰" + 0.037*"上市" + 0.025*"普拉多" + 0.025*"全新" + 0.025*"大众" + 0.025*"上汽" + 0.025*"上海大众" + 0.025*"SUV"'),
 (2,
  '0.037*"丰田" + 0.037*"普拉多" + 0.037*"柴油" + 0.037*"认输" + 0.037*"长城" + 0.037*"这下" + 0.037*"发飙" + 0.037*"霸道" + 0.037*"版哈弗" + 0.037*"H9"'),
 (3,
  '0.045*"福特" + 0.024*"类似" + 0.024*"电流" + 0.024*"高频" + 0.024*"滋滋" + 0.024*"听到" + 0.024*"细微" + 0.024*"求解" + 0.024*"2.3" + 0.024*"启动"'),
 (4,
  '0.053*"什么" + 0.052*"R9" + 0.052*"跑步机" + 0.028*"牌子" + 0.028*"机油" + 0.028*"标号" + 0.028*"3.0" + 0.028*"大切" + 0.028*"取代" + 0.028*"汽油车"'),
 (5,
  '0.041*"怎么" + 0.041*"丰田" + 0.041*"普拉多" + 0.022*"很多" + 0.022*"导航" + 0.022*"请问" + 0.022*"升级" + 0.022*"老路" + 0.022*"神器" + 0.022*"2017"'),
 (6,
  '0.020*"普拉多" + 0.020*"中东" + 0.020*"多少" + 0.020*"四驱" + 0.020*"版实" + 0.020*"2.3" + 0.020*"全时" + 0.020*"拍实" + 0.020*"还是" + 0.020*"全新"')